# Fine tune for dialogue summarization

To improve the inferences, I will use a full fine-tuning approach on a FLAN-T5 model, which provides a high quality instruction tuned model and can summarize text. Then I will evaluate the results with ROUGE metrics.

## 1. Set up 

### Install packages and import components

In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

                                              0.0/2.1 MB ? eta -:--:--
     ---                                      0.2/2.1 MB 3.7 MB/s eta 0:00:01
     -------                                  0.4/2.1 MB 4.6 MB/s eta 0:00:01
     ----------                               0.5/2.1 MB 4.3 MB/s eta 0:00:01
     ----------------                         0.9/2.1 MB 5.0 MB/s eta 0:00:01
     ---------------------                    1.1/2.1 MB 6.1 MB/s eta 0:00:01
     ----------------------                   1.2/2.1 MB 4.6 MB/s eta 0:00:01
     -----------------------------            1.5/2.1 MB 5.2 MB/s eta 0:00:01
     -------------------------------------    1.9/2.1 MB 5.9 MB/s eta 0:00:01
     ---------------------------------------  2.1/2.1 MB 5.8 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Note: 

ERROR: Could not find a version that satisfies the requirement torch==1.13.1 (from versions: 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.13.1


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate

### Dataset

 DialogSum Hugging Face dataset: https://huggingface.co/datasets/knkarthick/dialogsum

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/Lenovo/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-931380d0e19583fc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

### FLAN-T5 Model

https://huggingface.co/docs/transformers/model_doc/flan-t5   This is the pre-trained FLAN-T5 model and its tokenizer  from HuggingFace

In [4]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16) #small version of FLAN-T5 by seeting torch_dtype
tokenizer = AutoTokenizer.from_pretrained(model_name)

C:\Users\Lenovo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## 2. Try model with zero shot inference

In [6]:
index = 150

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))

print(f'INPUT PROMPT:\n{prompt}')

print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

INPUT PROMPT:
Summarize the following conversation.

#Person1#: Taxi!
#Person2#: Where will you go, sir?
#Person1#: Friendship Hotel.
#Person2#: OK, it's not far from here.
#Person1#: I have something important to do, can you fast the speed?
#Person2#: Sure, I'll try my best. Here we are.
#Person1#: It's fast! How much should I pay you?
#Person2#: The reading on the meter is 15 yuan.
#Person1#: Here's 20 yuan, keep the change.
#Person2#: Thank you very much.

Summary:

BASELINE HUMAN SUMMARY:
#Person1# takes a taxi to the Friendship Hotel for something important.

MODEL GENERATION - ZERO SHOT:
The taxi will pick you up at the Friendship Hotel at 20 yuan.


Not looking good!

## 3. Full Fine-Tuning

### Get the output dataset ready for fine-tuning

 I need to convert the dialog-summary (prompt-response) pairs into  instructions for the LLM. To create the instruction I will create a function that use a format that looks like this:
1.Start the dialogue "summarize the following conversation"
 
 Training prompt (dialogue): 
 
    Summarize the following conversation.

    Person 1: ...
    Person 2: ...
    
2.Start the summary with "Summary"  
  Summary: 
  
    Training response (summary):
    
    Person 1 and Person 2 ...
    
    
Then preprocess the prompt-response dataset into tokens

In [7]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

In [8]:
# The dataset actually contains 3 diff splits: train, validation, test. So the tokenize_function code is handling all data across all splits .
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True) #just a sample

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
#I want to check the size of the train, test and validation datasets
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})


### Fine-Tune the Model:Pass the preprocessed dataset with reference to the original model
I will use the built-in Hugging Face Trainer class https://huggingface.co/docs/transformers/main_classes/trainer

In [11]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1, #change
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1 #change
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16) #an instance of the AutoModelForSeq2SeqLM class for the instruct model:

## 4. Evaluation with ROUGE Metric

In [ ]:
rouge = evaluate.load('rouge')

Generate the outputs for some  sample of the test dataset

In [ ]:
dialogues = dataset['test'][0:5]['dialogue']
human_baseline_summaries = dataset['test'][0:5]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)
    
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)